In [ ]:
%run components/util/set_env.ipynb

In [ ]:
import os
import v3io_frames as v3f
from v3io import dataplane
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

DASHBOARD_URL = f"https://dashboard.{os.getenv('IGZ_NAMESPACE_DOMAIN')}"
session = requests.Session()
session.auth = (os.getenv("V3IO_USERNAME"), os.getenv("IGZ_AUTH"))
auth = session.post(f'{DASHBOARD_URL}/api/sessions', verify=False)

# Remove API Gateway
try:
    print(f"Deleting API Gateway '{API_GATEWAY}'...")
    response = session.delete(f'{DASHBOARD_URL}/api/api_gateways/{API_GATEWAY}')
except:
    print(f"Failed to delete API Gateway '{API_GATEWAY}'.")
    pass

# Remove functions
functions = [f'{PROJECT}-{GET_IMAGE_FUNCTION}', f'{PROJECT}-{FACIAL_RECOGNITION_FUNCTION}']
for function in functions:
    try:
        print(f"Deleting Function '{function}'...")
        response = session.delete(f'{DASHBOARD_URL}/api/functions/{function}')
    except:
        print(f"Failed to delete Function '{function}'.")
        pass

# Remove streams
frames_client = v3f.Client("framesd:8081",container="bigdata")
streams = [RAW_VIDEO_STREAM, TAGGED_VIDEO_STREAM]
for stream in streams:
    try:
        print(f"Deleting Stream '{stream}'...")
        frames_client.delete("stream",table=stream)
    except:
        print(f"Failed to delete Stream '{stream}'.")
        pass

# Remove table
v3io_client = v3f.Client('framesd:8081',container=IGZ_CONTAINER)
try:
    print(f"Deleting Table '{CAMERA_LIST_TBL}'...")
    v3io_client.delete('kv', CAMERA_LIST_TBL)
except:
    print(f"Table '{CAMERA_LIST_TBL}' already deleted.")
    pass

print("Done.")